## **Optimal Delta Hedging**

In [1]:
# -*- coding: utf-8 -*-
"""
Created on May 14 12:13:00 2024

@author: Bradley

SABR Model Calibration & Optimal Delta Hedging on SPX500 Options
"""

'\nCreated on May 14 12:13:00 2024\n\n@author: Bradley\n\nSABR Model Calibration & Optimal Delta Hedging on SPX500 Options\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from datetime import datetime
import os

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from pysabr import Hagan2002NormalSABR
from pysabr import Hagan2002LognormalSABR

import warnings
warnings.filterwarnings('ignore')  

from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important; }</style>"))

from IPython.display import display
pd.set_option('expand_frame_repr', True) 
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 180) 
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6) 
%config InlineBackend.figure_format = 'svg'
%matplotlib inline


### **SABR Model**

Recall the SABR model is given by
\begin{align*}
dF(t) &= \sigma(t)(F(t)+\theta)^{\beta}dW_1(t), F(0)=f\\
d\sigma(t) &= \nu\sigma(t)dW_2(t), \sigma(0)=\sigma_0\\
\end{align*}

where $W_1(t)$ and $W_2(t)$ are two correlated Wiener processes with correlation $\rho$, namely, $dW_1(t)dW_2(t) = \rho dt$.

- F: forward rate
- $\sigma$: volatility of forward rate
- $\nu$: volatility of volatility (volvol)
- $\theta$: shift parameter to avoid negative rates




In this project, we complete the following tasks:
1. Calibrate SABR model parameters on SPX 500 option market data
2. Calculate BS delta, SABR classic delta, and Bartlett's delta
3. Construct optimal delta hedging with Bartlett's delta
4. Compare hedging performance using hedging Gain (Hull, J., and White, 2016)

### **SABR Model Calibration**

We first calibrate the SABR model and calculate various deltas for SPX 500 option market data on a single day (2023-02-28).

In [3]:
import yfinance as yf

SPX_price = yf.Ticker("^spx").history(start="2023-02-28", end="2023-03-01", interval="1d")['Close'].values[0]
SPX_price

3970.14990234375

In [4]:
csv_file = "data/option20230228.csv"
csv_data = pd.read_csv(csv_file, low_memory = False)
option_df = pd.DataFrame(csv_data)

option_df.dropna(axis='index', how='all', subset=['impl_volatility'], inplace=True)
# Only look at the Call options here
option_df = option_df.loc[option_df['cp_flag'] == 'C']

option_df['date'] = option_df['date'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
option_df['exdate'] = option_df['exdate'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
option_df['Days_to_expiration'] = option_df['exdate'] - option_df['date']
option_df['strike_price'] = option_df['strike_price'] / 1000
option_df[['date', 'exdate', 'Days_to_expiration', 'strike_price', 'impl_volatility']].sort_values(by=['exdate', 'strike_price'])

,date,exdate,Days_to_expiration,strike_price,impl_volatility
6374,2023-02-28,2023-03-01,1 days,3000.0,2.939785
6375,2023-02-28,2023-03-01,1 days,3100.0,2.645950
6376,2023-02-28,2023-03-01,1 days,3200.0,2.358092
6377,2023-02-28,2023-03-01,1 days,3250.0,2.216078
6378,2023-02-28,2023-03-01,1 days,3300.0,2.075151
...,...,...,...,...,...
5685,2023-02-28,2027-12-17,1753 days,8800.0,0.154445
5686,2023-02-28,2027-12-17,1753 days,9200.0,0.156795
5687,2023-02-28,2027-12-17,1753 days,9600.0,0.159095
5636,2023-02-28,2027-12-17,1753 days,10000.0,0.161302


In [5]:
option_df['Days_to_expiration'].describe()

count                           8818
mean     118 days 12:30:42.050351554
std      212 days 23:09:49.301582572
min                  1 days 00:00:00
25%                 17 days 00:00:00
50%                 52 days 00:00:00
75%                122 days 00:00:00
max               1753 days 00:00:00
Name: Days_to_expiration, dtype: object

We have options with maturity ranging from 1 day to 5 years. Now we plot the implied volatility curve for a particular maturity.

In [6]:
# An example of the option data of 1753d maturity
mat = 17
print('Maturity =', mat)

option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]

data = dict(zip(option_day_impvol['strike_price'],option_day_impvol['impl_volatility']))
df = pd.DataFrame(data, index=[0]).T.reset_index()
df.rename(columns={'index':'K', 0:mat}, inplace=True)

df = df.sort_values(by='K',ascending=True).reset_index(drop=True)

# plot implied vol smile
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['K'], y=df[mat], mode='lines', name='Impvol Smile'))
fig.update_layout(title=f"Impvol smile, Expiration Day={mat}d, 20230228", xaxis_title='Strike Price', yaxis_title='Implied Volatility', width=800, height=400)
iplot(fig)

Maturity = 17


The graph shows a nice smile shape, which is a typical feature of the implied volatility curve. Now we calibrate the SABR model to the SPX 500 option market data. For equity options, it is proper to use SABR lognormal implied volatility with $\beta=1$ is suggested in the literature. Here, we calibrate the SABR model with $\beta=1$ using two methods for cross-checking:
1. non-linear least square method
2. pysabr library

#### **Calibration using pysabr package**

In [7]:
# normal model calibration

from pysabr import Hagan2002LognormalSABR

maturity = sorted(list(option_df['Days_to_expiration'].dt.days.unique()))

param_LN = pd.DataFrame(index=maturity, columns=['sigma0', 'rho', 'volvol'], dtype=float)
param_LN.index.name = 'Maturity'


# fit SABR model for each maturity
for mat in maturity:
    if mat == 0: continue
    
    sabr = Hagan2002LognormalSABR(f=SPX_price, shift=0, t=mat/365, beta=1) # fix beta=1
    
    option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]
    # only use options with strike near the forward price
    option_day_impvol = option_day_impvol.loc[abs(option_day_impvol['strike_price'] - SPX_price) < 0.05 * SPX_price]
    
    K_array = option_day_impvol['strike_price'].to_numpy() 
    # print(K_array)    
    vol_array = option_day_impvol['impl_volatility'].to_numpy() 
    # print(vol_array)
    [sigma0, rho, volvol] = sabr.fit(K_array, vol_array)
    param_LN.loc[mat] = [sigma0, rho, volvol]
param_LN.head(30)


,sigma0,rho,volvol
Maturity,,,
1,0.002420,-0.862427,0.270303
2,0.002314,-0.788049,0.150775
3,0.002207,-0.727742,0.112246
6,0.001752,-0.678295,0.070819
7,0.001892,-0.646203,0.057573
8,0.001890,-0.632558,0.051953
9,0.001905,-0.638101,0.047219
10,0.002035,-0.649829,0.040573
13,0.001869,-0.661143,0.034313


#### **Calibration using NLS algorithm**

In [8]:
# Log-normal model

from scipy.optimize import curve_fit

# For ease of optimization, we vectorize the SABR lognormal volatility function regarding strike K and change parameter order
def sabr_lognormal_vol(K, sigma0, rho, volvol, f, t, beta=1):
    """
    Returns: Lognormal implied volatility as per the SABR model (without shift)
    """
    # Vectorized condition to check if f is approximately equal to K
    close_to_atm = np.abs(f - K) < 1e-4 
    
    # ATM case calculations
    A_atm = (beta-1)**2 * sigma0**2 / (24 * f**(2-2*beta))
    B_atm = rho * sigma0 * volvol * beta / (4 * f**(1-beta))
    C_atm = (2-3*rho**2) * volvol**2 / 24
    sigma_LN_atm = sigma0 * f**(beta-1) * (1 + (A_atm + B_atm + C_atm) * t)

    # General case calculations
    if beta != 1:
        zeta_K = (volvol / (sigma0 * (1-beta))) * (f**(1-beta) - K**(1-beta))
    else:
        zeta_K = volvol * np.log(f/K) / sigma0
    x_zeta = (1 / volvol) * np.log((np.sqrt(1 - 2 * rho * zeta_K + zeta_K**2) + zeta_K - rho) / (1-rho))
    A = (1/24) * (beta-1)**2 * sigma0**2 * f**(beta-1) * K**(beta-1)
    B = (1/4) * rho * volvol * sigma0 * f**((beta-1)/2) * K**((beta-1)/2)
    C = (1/24) * (2-3*rho**2) * volvol**2
    sigma_LN = np.log(f/K) / x_zeta * (1 + (A + B + C) * t)

    # Combine ATM and non-ATM cases using np.where
    sigma_LN = np.where(close_to_atm, sigma_LN_atm, sigma_LN)
    return sigma_LN * 100

maturity = sorted(list(option_df['Days_to_expiration'].dt.days.unique()))
param_LN = pd.DataFrame(index=maturity, columns=['sigma0', 'rho', 'volvol'], dtype=float)
param_LN.index.name = 'Maturity'

for mat in sorted(list(option_df['Days_to_expiration'].dt.days.unique())):
    # Neglect the option that expires today
    if mat == 0:
        continue
    
    initial_guess = [0.1, 0.0, 0.3]  # sigma0, rho, volvol
    
    option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]
    # only use options with strike near the forward price
    option_day_impvol = option_day_impvol.loc[abs(option_day_impvol['strike_price'] - SPX_price) < 0.05 * SPX_price]
    
    K_array = option_day_impvol['strike_price'].to_numpy() 
    vol_array = option_day_impvol['impl_volatility'].to_numpy() 
    
    
    bounds = ([0, -1, 0], [1, 1, 2])
    
    params, covariance = curve_fit(lambda K, sigma0, rho, volvol: sabr_lognormal_vol(K, sigma0, rho, volvol, SPX_price, mat/365), 
                                   xdata=K_array, ydata=vol_array, p0=initial_guess, maxfev=5000, bounds=bounds)
    
    param_LN.loc[mat] = params
param_LN.head(30)

,sigma0,rho,volvol
Maturity,,,
1,0.002420,-0.862426,0.270301
2,0.002314,-0.788050,0.150773
3,0.002207,-0.727743,0.112245
6,0.001752,-0.678299,0.070818
7,0.001892,-0.646207,0.057573
8,0.001890,-0.632564,0.051952
9,0.001905,-0.638107,0.047218
10,0.002035,-0.649839,0.040573
13,0.001869,-0.661159,0.034312


#### **Another Version**

In [9]:
# Log-normal model

from scipy.optimize import curve_fit


def sabr_implied_vol(K, vol, rho, volvol, S, T, r=0, q=0, beta=1):

    F = S * np.exp((r - q) * T)
    x = (F * K) ** ((1 - beta) / 2)
    y = (1 - beta) * np.log(F / K)
    A = vol / (x * (1 + y * y / 24 + y * y * y * y / 1920))
    B = 1 + T * (
        ((1 - beta) ** 2) * (vol * vol) / (24 * x * x)
        + rho * beta * volvol * vol / (4 * x)
        + volvol * volvol * (2 - 3 * rho * rho) / 24
    )
    Phi = (volvol * x / vol) * np.log(F / K)
    Chi = np.log((np.sqrt(1 - 2 * rho * Phi + Phi * Phi) + Phi - rho) / (1 - rho))

    SABRIV = np.where(F == K, vol * B / (F ** (1 - beta)), A * B * Phi / Chi)

    return SABRIV * 100


maturity = sorted(list(option_df['Days_to_expiration'].dt.days.unique()))
param_LN = pd.DataFrame(index=maturity, columns=['sigma0', 'rho', 'volvol'], dtype=float)
param_LN.index.name = 'Maturity'

for mat in sorted(list(option_df['Days_to_expiration'].dt.days.unique())):
    # Neglect the option that expires today
    if mat == 0:
        continue
    
    initial_guess = [0.1, 0.0, 0.3]  # sigma0, rho, volvol
    
    option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]
    # only use options with strike near the forward price
    option_day_impvol = option_day_impvol.loc[abs(option_day_impvol['strike_price'] - SPX_price) < 0.05 * SPX_price]
    
    K_array = option_day_impvol['strike_price'].to_numpy() 
    vol_array = option_day_impvol['impl_volatility'].to_numpy() 
    
    
    bounds = ([0, -1, 0], [1, 1, 2])
    
    params, covariance = curve_fit(lambda K, sigma0, rho, volvol: sabr_implied_vol(K, sigma0, rho, volvol, SPX_price, mat/365), 
                                   xdata=K_array, ydata=vol_array, p0=initial_guess, maxfev=5000, bounds=bounds)
    
    param_LN.loc[mat] = params
param_LN.head(30)

,sigma0,rho,volvol
Maturity,,,
1,0.002420,-0.862426,0.270301
2,0.002314,-0.788050,0.150773
3,0.002207,-0.727743,0.112245
6,0.001752,-0.678299,0.070818
7,0.001892,-0.646207,0.057573
8,0.001890,-0.632564,0.051952
9,0.001905,-0.638107,0.047218
10,0.002035,-0.649839,0.040573
13,0.001869,-0.661159,0.034312


Results are exactly the same.

We see our optimization results are consistent with the built-in package results.

The result is also consistent with the empirical properties:
- The vol in vol parameter shows a persistent stable term structure and is monotonically decreasing with maturity

Let's check some of our fitting results on the implied volatility smile curves for near-the-money options.

In [10]:
for mat in [3,6,8,10,20,45,479]:

    sigma0, rho, volvol = param_LN.loc[mat]

    option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]
    # check for near ATM options
    option_day_impvol = option_day_impvol.loc[abs(option_day_impvol['strike_price'] - SPX_price) < 0.05 * SPX_price]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=option_day_impvol['strike_price'], y=option_day_impvol['impl_volatility'], mode='markers', name='Market Impvol'))
    fig.add_trace(go.Scatter(x=option_day_impvol['strike_price'], y=sabr_lognormal_vol(option_day_impvol['strike_price'], sigma0, rho, volvol, SPX_price, mat/365), mode='lines', name='SABR Impvol'))
    fig.update_layout(title=f"Lognormal SABR Calibration, Maturity={mat}d, 20230228", xaxis_title='Strike Price', yaxis_title='Implied Volatility', width=800, height=400)

    iplot(fig)

Our calibrated SABR model fits the implied volatility very well across all maturities.

In [11]:
# Implied Volatility Smile for different maturities and beta

for mat in [3, 10, 20]:
    fig = go.Figure()
    for beta in [0, 0.5, 1]:
        # calibrate the model again
        sabr = Hagan2002LognormalSABR(f=SPX_price, shift=0, t=mat/365, beta=beta) # fix beta=1
        option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]
        # Optional: only use options with strike near the forward price
        option_day_impvol = option_day_impvol.loc[abs(option_day_impvol['strike_price'] - SPX_price) < 0.1 * SPX_price]
        K_array = option_day_impvol['strike_price'].to_numpy() 
        vol_array = option_day_impvol['impl_volatility'].to_numpy() 
        # get fitted parameters
        sigma0, rho, volvol = sabr.fit(K_array, vol_array)    
        fig.add_trace(go.Scatter(x=option_day_impvol['strike_price'], y=sabr_lognormal_vol(option_day_impvol['strike_price'], sigma0, rho, volvol, SPX_price, mat/365, beta=beta), mode='lines', name=f'beta={beta}'))

    fig.update_layout(title=f"Implied Vol Fit, Maturity={mat}d, 20230228", xaxis_title='Strike Price', yaxis_title='Delta', width=800, height=400)

    iplot(fig)

The calibration goes well for different maturities and beta choices.

### **Delta Calculations**

BS delta is given by $\Delta_{BS} = N(d_1)$, where $d_1 = \cfrac{\log(F/K) + \frac{1}{2}\sigma^2T}{\sigma\sqrt{T}}$. The SABR delta is given by
$$\Delta^{\text{SABR}} = \frac{\partial B}{\partial F} + \frac{\partial B}{\partial \sigma}  \frac{\partial \sigma_{\text{imp}}}{\partial F}  $$

The Bartlett's delta further incorporates the adjustment for the implied volatility skew

$$\Delta^{\text{Bartlett}} = \frac{\partial B}{\partial F} + \frac{\partial B}{\partial \sigma} \left( \frac{\partial \sigma_{\text{imp}}}{\partial F} + \frac{\partial \sigma_{\text{imp}}}{\partial \sigma} \frac{\rho \alpha}{C(F_t)}\right) $$

It is shown in Hagan (2019) that the Bartlett's delta is the optimal delta for hedging in the SABR model, which can be approximated by
$$\Delta^{mod}\approx \Delta^{BS}+\text{Vega}^{BS}\times \eta$$


In [12]:
from scipy.stats import norm

def sabr_implied_vol(vol, T, S, K, r, q, beta, volvol, rho):

    F = S * np.exp((r - q) * T)
    x = (F * K) ** ((1 - beta) / 2)
    y = (1 - beta) * np.log(F / K)
    A = vol / (x * (1 + y * y / 24 + y * y * y * y / 1920))
    B = 1 + T * (
        ((1 - beta) ** 2) * (vol * vol) / (24 * x * x)
        + rho * beta * volvol * vol / (4 * x)
        + volvol * volvol * (2 - 3 * rho * rho) / 24
    )
    Phi = (volvol * x / vol) * np.log(F / K)
    Chi = np.log((np.sqrt(1 - 2 * rho * Phi + Phi * Phi) + Phi - rho) / (1 - rho))

    SABRIV = np.where(F == K, vol * B / (F ** (1 - beta)), A * B * Phi / Chi)

    return SABRIV * 100

def bs_call(iv, T, S, K, r, q):
    d1 = (np.log(S / K) + (r - q + iv * iv / 2) * T) / (iv * np.sqrt(T))
    d2 = d1 - iv * np.sqrt(T)
    bs_price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return bs_price


def bs_delta(sigma, T, S, K, r, q):
    iv = sigma
    d1 = (np.log(S / K) + (r - q + iv * iv / 2) * T) / (iv * np.sqrt(T))
    bs_delta = np.exp(-q * T) * norm.cdf(d1)
    return bs_delta

# def sabr_delta(sigma0, T, S, K, r, q, beta, rho, volvol):
#     iv = sabr_implied_vol(sigma0, T, S, K, r, q, beta, volvol, rho)
#     d1 = (np.log(S / K) + (r - q + iv * iv / 2) * T) / (iv * np.sqrt(T))
#     bs_delta = np.exp(-q * T) * norm.cdf(d1)
#     return bs_delta


# def sabr_barlette_delta(sigma, T, S, K, r, q, ds, beta, volvol, rho):
#     dsigma = ds * volvol * rho / (S ** beta)
#     vol1 = sabr_implied_vol(sigma, T, S, K, r, q, beta, volvol, rho)
#     vol2 = sabr_implied_vol(sigma + dsigma, T, S + ds, K, r, q, beta, volvol, rho)
#     bs_price1 = bs_call(vol1, T, S, K, r, q)
#     bs_price2 = bs_call(vol2, T, S+ds, K, r, q)
#     b_delta = (bs_price2 - bs_price1) / ds

#     # avoid numerical issues like -0.0001 for delta values
#     return np.maximum(b_delta, 0)

def sabr_delta(sigma, T, S, K, r, q, beta, volvol, rho):
    sabr_iv = sabr_implied_vol(sigma, T, S, K, r, q, beta, volvol, rho)
    ds = 1e-6 * S
    dsigma = 1e-6 * sigma
    dsigma_ds = (sabr_implied_vol(sigma, T, S+ds, K, r, q, beta, volvol, rho) - sabr_implied_vol(sigma, T, S-ds, K, r, q, beta, volvol, rho)) / ds / 2
    dsigma_dsigma0 = (sabr_implied_vol(sigma+dsigma, T, S, K, r, q, beta, volvol, rho) - sabr_implied_vol(sigma-dsigma, T, S, K, r, q, beta, volvol, rho)) / dsigma / 2
    d = (np.log(S/K)+0.5*sabr_iv**2*T) / (sabr_iv * np.sqrt(T))
    return norm.cdf(d) + np.sqrt(T) * norm.pdf(d) * (dsigma_ds)


def sabr_barlette_delta(sigma, T, S, K, r, q, ds, beta, volvol, rho):
    sabr_iv = sabr_implied_vol(sigma, T, S, K, r, q, beta, volvol, rho)
    ds = 1e-6 * S
    dsigma = 1e-6 * sigma
    dsigma_ds = (sabr_implied_vol(sigma, T, S+ds, K, r, q, beta, volvol, rho) - sabr_implied_vol(sigma, T, S-ds, K, r, q, beta, volvol, rho)) / ds / 2
    dsigma_dsigma0 = (sabr_implied_vol(sigma+dsigma, T, S, K, r, q, beta, volvol, rho) - sabr_implied_vol(sigma-dsigma, T, S, K, r, q, beta, volvol, rho)) / dsigma / 2
    d = (np.log(S/K)+0.5*sabr_iv**2*T) / (sabr_iv * np.sqrt(T))
    return norm.cdf(d) + np.sqrt(T) * norm.pdf(d) * (dsigma_ds+dsigma_dsigma0*rho*volvol/(S**beta))


In [13]:
# test a single maturity case with beta = 1
mat = 1
sigma0, rho, volvol = param_LN.loc[mat]
f = SPX_price
t = mat/365
option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]
# Optional: do a filter to only visualize options with strike near the forward price
option_day_impvol = option_day_impvol.loc[abs(option_day_impvol['strike_price'] - SPX_price) < 0.2 * SPX_price]

option_day_impvol['BS Delta'] = bs_delta(option_day_impvol['impl_volatility'], t, f, option_day_impvol['strike_price'], 0, 0)
option_day_impvol['SABR Delta'] = sabr_delta(sigma0, t, f, option_day_impvol['strike_price'], 0, 0, beta=1, volvol=volvol, rho=rho)
option_day_impvol['Barlette Delta'] = sabr_barlette_delta(sigma0, t, f, option_day_impvol['strike_price'], 0, 0, ds=0.1, beta=1, volvol=volvol, rho=rho)
# print(option_day_impvol)

fig = go.Figure()
fig.add_trace(go.Scatter(x=option_day_impvol['strike_price'], y=option_day_impvol['BS Delta'], mode='markers', name='BS Delta'))
fig.add_trace(go.Scatter(x=option_day_impvol['strike_price'], y=option_day_impvol['SABR Delta'], mode='lines', name='SABR Delta'))
fig.add_trace(go.Scatter(x=option_day_impvol['strike_price'], y=option_day_impvol['Barlette Delta'], mode='lines', name='Barlette Delta'))
fig.update_layout(title=f"Delta Calibration, Maturity={mat}d, 20230228", xaxis_title='Strike Price', yaxis_title='Delta', width=1000, height=600)

iplot(fig)

SABR delta and Bartlett delta are very close, except that **Bartlett delta incorporates the adjustment for the implied volatility skew**. We now plot the two deltas of the options against the strike price for a given maturity with different betas.

In [14]:
# Plot results of different maturities on different subplots

for mat in [1, 6, 10, 20, 45]:
    fig = go.Figure()
    
    for beta in [0, 0.5, 1]:
        # calibrate the model again
        sabr = Hagan2002LognormalSABR(f=SPX_price, shift=0, t=mat/365, beta=beta) # fix beta=1
        option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]
        # Optional: only use options with strike near the forward price
        option_day_impvol = option_day_impvol.loc[abs(option_day_impvol['strike_price'] - SPX_price) < 0.2 * SPX_price]
        K_array = option_day_impvol['strike_price'].to_numpy() 
        vol_array = option_day_impvol['impl_volatility'].to_numpy() 
        # get fitted parameters
        sigma0, rho, volvol = sabr.fit(K_array, vol_array)    
        # print("beta:", beta, "sigma0:", sigma0, "rho:", rho, "volvol:", volvol)
        # calculate the delta
        sabrdelta = sabr_delta(sigma0, t, f, option_day_impvol['strike_price'], 0, 0, beta=beta, volvol=volvol, rho=rho)
        fig.add_trace(go.Scatter(x=option_day_impvol['strike_price'], y=sabrdelta, mode='lines', name=f'beta={beta}'))

    fig.update_layout(title=f"SABR Delta, Maturity={mat}d, 20230228", xaxis_title='Strike Price', yaxis_title='Delta', width=800, height=400)

    iplot(fig)

In [15]:
# Barlette Delta for different beta values

for mat in [1, 6, 10, 20, 45]:
    fig = go.Figure()
    
    for beta in [0, 0.5, 0.8]:
        # calibrate the model again
        sabr = Hagan2002LognormalSABR(f=SPX_price, shift=0, t=mat/365, beta=beta) # fix beta=1
        option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]
        # Optional: only use options with strike near the forward price
        option_day_impvol = option_day_impvol.loc[abs(option_day_impvol['strike_price'] - SPX_price) < 0.1 * SPX_price]
        K_array = option_day_impvol['strike_price'].to_numpy() 
        vol_array = option_day_impvol['impl_volatility'].to_numpy() 
        # get fitted parameters
        sigma0, rho, volvol = sabr.fit(K_array, vol_array)    
        # print("beta:", beta, "sigma0:", sigma0, "rho:", rho, "volvol:", volvol)
        # calculate the delta
        barlette_delta = sabr_barlette_delta(sigma0, t, f, option_day_impvol['strike_price'], 0, 0, ds=1, beta=beta, volvol=volvol, rho=rho)
        fig.add_trace(go.Scatter(x=option_day_impvol['strike_price'], y=barlette_delta, mode='lines', name=f'beta={beta}'))

    fig.update_layout(title=f"Barlette Delta, Maturity={mat}d, 20230228", xaxis_title='Strike Price', yaxis_title='Delta', width=800, height=400)

    iplot(fig)

SABR delta may lead to different conventional hedges especially near the money. Bartlett’s delta is nearly independent of the beta choice and varies as the option maturity changes. It is usually showed that Bartlett's deltas tends to provide more robust hedges.

In [16]:
# Plot results of different maturities on the same plot

fig = go.Figure()

for mat in [3, 10, 45, 92, 479]:
    for beta in [0.5, 1]:
        # calibrate the model again
        sabr = Hagan2002LognormalSABR(f=SPX_price, shift=0, t=mat/365, beta=beta) # fix beta=1
        option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]
        # Optional: only use options with strike near the forward price
        option_day_impvol = option_day_impvol.loc[abs(option_day_impvol['strike_price'] - SPX_price) < 0.05 * SPX_price]
        K_array = option_day_impvol['strike_price'].to_numpy() 
        vol_array = option_day_impvol['impl_volatility'].to_numpy() 
        # get fitted parameters
        sigma0, rho, volvol = sabr.fit(K_array, vol_array)    
        # calculate the delta
        sabrdelta = sabr_delta(sigma0, t, f, option_day_impvol['strike_price'], 0, 0, beta=beta, volvol=volvol, rho=rho)
        fig.add_trace(go.Scatter(x=option_day_impvol['strike_price'], y=sabrdelta, mode='lines', name=f'beta={beta}, Maturity={mat}d'))

    fig.update_layout(title=f"SABR Delta, 20230228", xaxis_title='Strike Price', yaxis_title='Delta', width=1000, height=600)

iplot(fig)

In [17]:
# Plot results of different maturities on the same plot

fig = go.Figure()

for mat in [3, 10, 45, 92, 479]:
    for beta in [0.5, 1]:
        # calibrate the model again
        sabr = Hagan2002LognormalSABR(f=SPX_price, shift=0, t=mat/365, beta=beta) # fix beta=1
        option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]
        # Optional: only use options with strike near the forward price
        option_day_impvol = option_day_impvol.loc[abs(option_day_impvol['strike_price'] - SPX_price) < 0.05 * SPX_price]
        K_array = option_day_impvol['strike_price'].to_numpy() 
        vol_array = option_day_impvol['impl_volatility'].to_numpy() 
        # get fitted parameters
        sigma0, rho, volvol = sabr.fit(K_array, vol_array)    
        # calculate the delta
        barlette_delta = sabr_barlette_delta(sigma0, t, f, option_day_impvol['strike_price'], 0, 0, ds=1, beta=beta, volvol=volvol, rho=rho)
        fig.add_trace(go.Scatter(x=option_day_impvol['strike_price'], y=barlette_delta, mode='lines', name=f'beta={beta}, Maturity={mat}d'))

    fig.update_layout(title=f"Barlette Delta, 20230228", xaxis_title='Strike Price', yaxis_title='Delta', width=1000, height=600)

iplot(fig)

### **Optimal Hedging with Bartlett's Delta**

Now we calibrate the SABR model and calculate the optimal delta hedging using Bartlett's delta for SPX 500 time series data from WRDS.

In [18]:
import yfinance as yf

SPX_price_series = yf.Ticker("^spx").history(start="2023-02-01", end="2023-03-01", interval="1d")['Close']
SPX_price_series

Date
2023-02-01    4119.209961
2023-02-02    4179.759766
2023-02-03    4136.479980
2023-02-06    4111.080078
2023-02-07    4164.000000
2023-02-08    4117.859863
2023-02-09    4081.500000
2023-02-10    4090.459961
2023-02-13    4137.290039
2023-02-14    4136.129883
2023-02-15    4147.600098
2023-02-16    4090.409912
2023-02-17    4079.090088
2023-02-21    3997.340088
2023-02-22    3991.050049
2023-02-23    4012.320068
2023-02-24    3970.040039
2023-02-27    3982.239990
2023-02-28    3970.149902
Name: Close, dtype: float64

In [19]:
SPX_price_diff = yf.Ticker("^spx").history(start="2023-02-01", end="2023-03-02", interval="1d")['Close'].diff().shift(-1).dropna()
SPX_price_diff

Date
2023-02-01    60.549805
2023-02-02   -43.279785
2023-02-03   -25.399902
2023-02-06    52.919922
2023-02-07   -46.140137
2023-02-08   -36.359863
2023-02-09     8.959961
2023-02-10    46.830078
2023-02-13    -1.160156
2023-02-14    11.470215
2023-02-15   -57.190186
2023-02-16   -11.319824
2023-02-17   -81.750000
2023-02-21    -6.290039
2023-02-22    21.270020
2023-02-23   -42.280029
2023-02-24    12.199951
2023-02-27   -12.090088
2023-02-28   -18.760010
Name: Close, dtype: float64

In [20]:
data = pd.read_csv('data/option20230201_20230228.csv')
data.dropna(axis='index', how='all', subset=['impl_volatility'], inplace=True)
data = data.loc[data['cp_flag'] == 'C']
data['date'] = data['date'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
data['exdate'] = data['exdate'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
data['Days_to_expiration'] = data['exdate'] - data['date']
data['strike_price'] = data['strike_price'] / 1000
data

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag,Days_to_expiration
121,108105,2023-02-01,SPX 230217C3595000,1,2023-02-17,NaN,C,3595.0,522.5,533.3,...,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN,16 days
125,108105,2023-02-01,SPX 230217C3615000,1,2023-02-17,NaN,C,3615.0,502.7,513.5,...,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN,16 days
127,108105,2023-02-01,SPX 230217C3625000,1,2023-02-17,2023-01-25,C,3625.0,490.5,505.5,...,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN,16 days
129,108105,2023-02-01,SPX 230217C3635000,1,2023-02-17,2023-01-31,C,3635.0,480.6,495.6,...,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN,16 days
130,108105,2023-02-01,SPX 230217C3640000,1,2023-02-17,2022-10-14,C,3640.0,475.6,490.3,...,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN,16 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175296,108105,2023-02-28,SPXW 231229C5100000,1,2023-12-29,2023-02-28,C,5100.0,8.7,9.0,...,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN,304 days
175297,108105,2023-02-28,SPXW 231229C5200000,1,2023-12-29,2023-02-24,C,5200.0,6.1,6.5,...,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN,304 days
175298,108105,2023-02-28,SPXW 231229C5300000,1,2023-12-29,2023-02-16,C,5300.0,4.5,4.9,...,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN,304 days
175299,108105,2023-02-28,SPXW 231229C5400000,1,2023-12-29,2023-02-28,C,5400.0,3.4,3.8,...,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN,304 days


In [21]:
data['Option'] = (data['best_bid'] + data['best_offer']) / 2
data['diff_Option'] = data.groupby('symbol')['Option'].diff().shift(-1)
data.dropna(subset=['diff_Option'], inplace=True)
data[['date', 'exdate', 'Days_to_expiration', 'strike_price', 'impl_volatility', 'Option', 'diff_Option']]

,date,exdate,Days_to_expiration,strike_price,impl_volatility,Option,diff_Option
9028,2023-02-01,2023-12-29,331 days,5400.0,0.129523,5.80,56.75
9200,2023-02-02,2023-02-17,15 days,3840.0,0.146858,343.50,56.15
9202,2023-02-02,2023-02-17,15 days,3850.0,0.188947,334.10,55.95
9203,2023-02-02,2023-02-17,15 days,3855.0,0.132911,328.50,56.60
9204,2023-02-02,2023-02-17,15 days,3860.0,0.193585,324.40,56.05
...,...,...,...,...,...,...,...
175295,2023-02-28,2023-12-29,304 days,5025.0,0.136739,11.60,-0.75
175296,2023-02-28,2023-12-29,304 days,5100.0,0.137016,8.85,-0.55
175297,2023-02-28,2023-12-29,304 days,5200.0,0.138055,6.30,-0.35
175298,2023-02-28,2023-12-29,304 days,5300.0,0.140171,4.70,-0.20


In [22]:
data['Days_to_expiration'].describe()

count                         141619
mean     131 days 17:50:38.566858966
std      224 days 13:17:14.502578776
min                  1 days 00:00:00
25%                 22 days 00:00:00
50%                 66 days 00:00:00
75%                143 days 00:00:00
max               1779 days 00:00:00
Name: Days_to_expiration, dtype: object

In [23]:
col_dates = [str(date)[:10] for date in sorted(list(data['date'].unique()))]

sigma0_LN = pd.DataFrame(columns=col_dates, dtype=float)
rho_LN = pd.DataFrame(columns=col_dates, dtype=float)
volvol_LN = pd.DataFrame(columns=col_dates, dtype=float)
sigma0_LN.index.name = 'Maturity'
rho_LN.index.name = 'Maturity'
volvol_LN.index.name = 'Maturity'

beta = 1

for mat in sorted(list(data['Days_to_expiration'].dt.days.unique())):
    
    if (mat < 1) or (mat > 200):
        continue
    
    for i, day in enumerate(sorted(list(data['date'].unique()))):
        option_df = data[data['date'] == day]
        SPX_price = SPX_price_series.loc[str(day)[:10]]
    
        option_day_impvol = option_df.loc[option_df['Days_to_expiration'].dt.days == mat].loc[:,['strike_price', 'impl_volatility']]

        # only fit the SABR model if there are enough options
        if len(option_day_impvol) < 100:
            continue

        K_array = option_day_impvol['strike_price'].to_numpy() 
        vol_array = option_day_impvol['impl_volatility'].to_numpy() 

        bounds = ([0, -1, 0], [1, 1, 2])
        params, covariance = curve_fit(lambda K, sigma0, rho, volvol: sabr_lognormal_vol(K, sigma0, rho, volvol, SPX_price, mat/365, beta=beta), 
                                    xdata=K_array, ydata=vol_array, p0=initial_guess, maxfev=5000, bounds=bounds)
    
        sigma0, rho, volvol = params

        sigma0_LN.loc[mat,str(day)[:10]] = sigma0
        rho_LN.loc[mat,str(day)[:10]] = rho
        volvol_LN.loc[mat,str(day)[:10]] = volvol
        

In [24]:
rho_LN.head(20)

,2023-02-01,2023-02-02,2023-02-03,2023-02-06,2023-02-07,2023-02-08,2023-02-09,2023-02-10,2023-02-13,2023-02-14,2023-02-15,2023-02-16,2023-02-17,2023-02-21,2023-02-22,2023-02-23,2023-02-24,2023-02-27,2023-02-28
Maturity,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,-0.931183,-0.934428,NaN,NaN,NaN,-0.163193,NaN,-0.329526,NaN,-0.009661,NaN,-0.928580,NaN,-0.765117,-0.968579
2,NaN,NaN,NaN,NaN,-0.930956,-0.882852,NaN,NaN,NaN,-0.797565,-0.706766,NaN,NaN,-0.665410,-0.385071,NaN,NaN,NaN,-0.958820
3,NaN,NaN,NaN,-0.410015,-0.785992,NaN,NaN,-0.334756,NaN,-0.611040,NaN,NaN,NaN,-0.630813,NaN,NaN,NaN,-0.250521,-0.964774
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.448493,-0.488103,NaN,NaN,NaN,NaN,NaN,NaN,-0.890116,-0.542955,-0.239174,NaN
5,NaN,NaN,NaN,NaN,NaN,-0.684957,NaN,-0.202179,NaN,NaN,NaN,-0.353460,NaN,NaN,NaN,-0.956349,NaN,NaN,NaN
6,NaN,NaN,-0.203523,NaN,-0.500877,-0.458180,NaN,-0.479177,NaN,NaN,NaN,-0.461941,-0.482583,-0.454067,-0.347352,-0.779615,-0.324296,NaN,-0.948033
7,NaN,NaN,-0.054266,NaN,-0.420612,-0.217879,-0.124261,-0.330128,NaN,-0.371396,NaN,-0.656009,-0.177585,-0.274286,NaN,-0.798245,-0.295912,-0.195363,-0.918967
8,NaN,NaN,NaN,-0.064332,-0.430219,-0.468815,-0.383328,NaN,NaN,-0.228369,NaN,-0.581930,NaN,-0.431980,-0.399518,-0.790687,NaN,-0.197275,-0.881203
9,NaN,NaN,NaN,-0.055299,-0.482086,-0.488476,NaN,NaN,NaN,-0.571284,-0.184006,NaN,NaN,-0.565420,-0.346144,NaN,NaN,-0.185949,-0.863109


In [25]:
volvol_LN.head(20)

,2023-02-01,2023-02-02,2023-02-03,2023-02-06,2023-02-07,2023-02-08,2023-02-09,2023-02-10,2023-02-13,2023-02-14,2023-02-15,2023-02-16,2023-02-17,2023-02-21,2023-02-22,2023-02-23,2023-02-24,2023-02-27,2023-02-28
Maturity,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,0.465389,0.500923,NaN,NaN,NaN,0.192484,NaN,0.299271,NaN,0.171503,NaN,0.470568,NaN,0.385879,0.469366
2,NaN,NaN,NaN,NaN,0.347890,0.325601,NaN,NaN,NaN,0.301804,0.306001,NaN,NaN,0.214893,0.147653,NaN,NaN,NaN,0.315464
3,NaN,NaN,NaN,0.190835,0.192596,NaN,NaN,0.478715,NaN,0.205439,NaN,NaN,NaN,0.168898,NaN,NaN,NaN,0.111954,0.268040
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.158216,0.131836,NaN,NaN,NaN,NaN,NaN,NaN,0.223727,0.128420,0.078690,NaN
5,NaN,NaN,NaN,NaN,NaN,0.209750,NaN,0.046407,NaN,NaN,NaN,0.080786,NaN,NaN,NaN,0.276911,NaN,NaN,NaN
6,NaN,NaN,0.060943,NaN,0.078315,0.126557,NaN,0.121142,NaN,NaN,NaN,0.064519,0.246556,0.085351,0.073933,0.157370,0.074007,NaN,0.187201
7,NaN,NaN,0.058476,NaN,0.058422,0.038254,0.041864,0.086784,NaN,0.078831,NaN,0.120893,0.052180,0.054325,NaN,0.143552,0.055971,0.054818,0.162735
8,NaN,NaN,NaN,0.041016,0.047524,0.109614,0.075999,NaN,NaN,0.093916,NaN,0.091011,NaN,0.044394,0.056478,0.135780,NaN,0.052493,0.155058
9,NaN,NaN,NaN,0.028292,0.055771,0.077938,NaN,NaN,NaN,0.116033,0.046201,NaN,NaN,0.075778,0.041099,NaN,NaN,0.036297,0.139161


In [26]:
# Example of calculating different delta time series for a particular maturity

mat = 20
sigma0 = sigma0_LN.loc[mat]
rho = rho_LN.loc[mat]
volvol = volvol_LN.loc[mat]

t = mat/365

option_day_impvol = data.loc[data['Days_to_expiration'].dt.days == mat].loc[:,['date', 'strike_price', 'impl_volatility', 'diff_Option', 'delta']]

option_day_impvol['SPX'] = option_day_impvol['date'].map(lambda x: SPX_price_series.loc[x])
option_day_impvol['diff_SPX'] = option_day_impvol['date'].map(lambda x: SPX_price_diff.loc[x])
option_day_impvol['sigma0'] = option_day_impvol['date'].map(lambda x: sigma0_LN.loc[mat, str(x)[:10]])
option_day_impvol['rho'] = option_day_impvol['date'].map(lambda x: rho_LN.loc[mat, str(x)[:10]])
option_day_impvol['volvol'] = option_day_impvol['date'].map(lambda x: volvol_LN.loc[mat, str(x)[:10]])
option_day_impvol['BS Delta'] = bs_delta(option_day_impvol['impl_volatility'], t, option_day_impvol['SPX'], option_day_impvol['strike_price'], 0, 0)
option_day_impvol['SABR Delta'] = sabr_delta(option_day_impvol['sigma0'], t, option_day_impvol['SPX'], option_day_impvol['strike_price'], 0, 0, beta=beta, volvol=option_day_impvol['volvol'], rho=option_day_impvol['rho'])
option_day_impvol['Barlette Delta'] = sabr_barlette_delta(option_day_impvol['sigma0'], t, option_day_impvol['SPX'], option_day_impvol['strike_price'], 0, 0, ds=0.1, beta=beta, volvol=option_day_impvol['volvol'], rho=option_day_impvol['rho'])
option_day_impvol.dropna(subset=['BS Delta', 'SABR Delta'], inplace=True)
option_day_impvol

,date,strike_price,impl_volatility,diff_Option,delta,SPX,diff_SPX,sigma0,rho,volvol,BS Delta,SABR Delta,Barlette Delta
42827,2023-02-07,3500.0,0.376226,29.350,0.978238,4164.000000,-46.140137,0.001593,-0.481880,0.033417,0.978128,0.987727,0.987724
42828,2023-02-07,3550.0,0.352687,29.500,0.976036,4164.000000,-46.140137,0.001593,-0.481880,0.033417,0.975787,0.984357,0.984353
42829,2023-02-07,3600.0,0.333930,55.800,0.971817,4164.000000,-46.140137,0.001593,-0.481880,0.033417,0.971350,0.979947,0.979942
42830,2023-02-07,3650.0,0.307244,55.250,0.969763,4164.000000,-46.140137,0.001593,-0.481880,0.033417,0.969107,0.974134,0.974128
42831,2023-02-07,3700.0,0.286204,54.900,0.964797,4164.000000,-46.140137,0.001593,-0.481880,0.033417,0.963829,0.966414,0.966407
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171509,2023-02-28,4500.0,0.182741,-0.025,0.002037,3970.149902,-18.760010,0.001430,-0.714787,0.082292,0.001824,0.025451,0.025440
171511,2023-02-28,4700.0,0.226475,0.000,0.000878,3970.149902,-18.760010,0.001430,-0.714787,0.082292,0.000798,0.017294,0.017285
171512,2023-02-28,4800.0,0.252402,0.000,0.000795,3970.149902,-18.760010,0.001430,-0.714787,0.082292,0.000729,0.014816,0.014808
171513,2023-02-28,5000.0,0.287131,0.000,0.000368,3970.149902,-18.760010,0.001430,-0.714787,0.082292,0.000340,0.011438,0.011431


In [27]:
# concatenate the delta data for all maturities

result = []

for mat in sigma0_LN.index:
    sigma0 = sigma0_LN.loc[mat]
    rho = rho_LN.loc[mat]
    volvol = volvol_LN.loc[mat]
    t = mat/365

    option_day_impvol = data.loc[data['Days_to_expiration'].dt.days == mat].loc[:,['date', 'strike_price', 'impl_volatility', 'diff_Option','Days_to_expiration', 'delta']]
    option_day_impvol['SPX'] = option_day_impvol['date'].map(lambda x: SPX_price_series.loc[x])
    option_day_impvol['diff_SPX'] = option_day_impvol['date'].map(lambda x: SPX_price_diff.loc[x])

    option_day_impvol['sigma0'] = option_day_impvol['date'].map(lambda x: sigma0_LN.loc[mat, str(x)[:10]])
    option_day_impvol['rho'] = option_day_impvol['date'].map(lambda x: rho_LN.loc[mat, str(x)[:10]])
    option_day_impvol['volvol'] = option_day_impvol['date'].map(lambda x: volvol_LN.loc[mat, str(x)[:10]])
    option_day_impvol['BS Delta'] = bs_delta(option_day_impvol['impl_volatility'], t, option_day_impvol['SPX'], option_day_impvol['strike_price'], 0, 0)
    option_day_impvol['SABR Delta'] = sabr_delta(option_day_impvol['sigma0'], t, option_day_impvol['SPX'], option_day_impvol['strike_price'], 0, 0, beta=beta, volvol=option_day_impvol['volvol'], rho=option_day_impvol['rho'])
    option_day_impvol['Barlette Delta'] = sabr_barlette_delta(option_day_impvol['sigma0'], t, option_day_impvol['SPX'], option_day_impvol['strike_price'], 0, 0, ds=0.1, beta=beta, volvol=option_day_impvol['volvol'], rho=option_day_impvol['rho'])
    
    option_day_impvol.drop(columns=['sigma0', 'rho', 'volvol', 'impl_volatility'], inplace=True)

    result.append(option_day_impvol)

result = pd.concat(result)
result.dropna(subset=['BS Delta', 'SABR Delta'], inplace=True)

result

,date,strike_price,diff_Option,Days_to_expiration,delta,SPX,diff_SPX,BS Delta,SABR Delta,Barlette Delta
40759,2023-02-07,3150.0,30.450,1 days,0.982493,4164.00000,-46.140137,0.982507,0.986387,0.986264
40760,2023-02-07,3200.0,30.450,1 days,0.981495,4164.00000,-46.140137,0.981508,0.985429,0.985303
40764,2023-02-07,3400.0,30.300,1 days,0.977343,4164.00000,-46.140137,0.977343,0.980447,0.980309
40765,2023-02-07,3450.0,30.300,1 days,0.976014,4164.00000,-46.140137,0.976009,0.978806,0.978665
40768,2023-02-07,3600.0,30.200,1 days,0.970910,4164.00000,-46.140137,0.970886,0.972357,0.972205
...,...,...,...,...,...,...,...,...,...,...
158714,2023-02-27,6400.0,0.050,200 days,0.000691,3982.23999,-12.090088,0.000441,0.000352,0.000352
158715,2023-02-27,6600.0,0.025,200 days,0.000655,3982.23999,-12.090088,0.000429,0.000241,0.000240
158716,2023-02-27,6800.0,0.000,200 days,0.000485,3982.23999,-12.090088,0.000320,0.000169,0.000169
158717,2023-02-27,7000.0,0.025,200 days,0.000464,3982.23999,-12.090088,0.000312,0.000122,0.000122


### **Delta Hedging Performance**

In Hull, J., and White (2016), the effectiveness of a hedge is measured by the $Gain$ metric, defined as the percentage reduction in the sum of squared residuals resulting from the hedge, i.e.
$$\text{Gain} = 1- \cfrac{\sum(\Delta f  - \delta_{\text{SABR}}\Delta S)^2}{\sum(\Delta f  - \delta_{\text{BS}}\Delta S)^2}$$
where SSE denotes sum of squared errors and BS delta serving as the benchmark. Using standard deviations rather than SSEs would produce a similar result but with a numerically smaller Gain. The following evaluation follows exactly the same procedure as in Hull, J., and White (2016).

**Important Note**
- Bucketing: Create nine moneyness buckets according to $\delta_{BS}$ and seven different option maturity buckets. Then calculate the Gain for each bucket.

- **SABR calibration performance would slightly worsen for very short-term maturities, since parameter calibration is volatile and near the money gamma is large. Hence it is a common practice to only consider calibrating options with decent maturities (e.g. > 14 days).**

In [28]:
# filter extreme delta
res = result.copy()

# according to Hull, J., and White (2016), filter out options with extreme delta values and maturity less than 14 days
res = res.loc[(res['delta'] > 0.05) & (res['delta'] < 0.95)]
res = res.loc[(res['Days_to_expiration'].dt.days > 14)]

# group the data by delta into 9 bins
res['bin1'] = pd.cut(res['delta'], bins=9, labels=False)
# further group the data by maturity into 7 bins
res['bin2'] = pd.cut(res['Days_to_expiration'].dt.days, bins=7, labels=False)
res

,date,strike_price,diff_Option,Days_to_expiration,delta,SPX,diff_SPX,BS Delta,SABR Delta,Barlette Delta,bin1,bin2
9217,2023-02-02,3925.0,51.75,15 days,0.949661,4179.759766,-43.279785,0.941462,0.932317,0.932313,8,0
9219,2023-02-02,3935.0,52.55,15 days,0.945420,4179.759766,-43.279785,0.936765,0.927315,0.927310,8,0
9220,2023-02-02,3940.0,51.70,15 days,0.940862,4179.759766,-43.279785,0.931813,0.924652,0.924647,8,0
9221,2023-02-02,3945.0,51.45,15 days,0.941517,4179.759766,-43.279785,0.932438,0.921873,0.921868,8,0
9222,2023-02-02,3950.0,51.00,15 days,0.937350,4179.759766,-43.279785,0.927914,0.918974,0.918969,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...
158691,2023-02-27,4725.0,-0.30,200 days,0.078226,3982.239990,-12.090088,0.055319,0.063032,0.063017,0,6
158692,2023-02-27,4750.0,-0.25,200 days,0.070284,3982.239990,-12.090088,0.049212,0.056735,0.056722,0,6
158693,2023-02-27,4775.0,-0.25,200 days,0.063056,3982.239990,-12.090088,0.043723,0.051045,0.051033,0,6
158694,2023-02-27,4800.0,-0.20,200 days,0.056585,3982.239990,-12.090088,0.038876,0.045916,0.045905,0,6


In [29]:
def calc_gain(df, sse=True):
    if sse:
        df['sse_bs'] = (df['diff_Option'] - df['delta']*df['diff_SPX'])**2
        df['sse_sabr'] = (df['diff_Option'] - df['SABR Delta']*df['diff_SPX'])**2
        df['sse_barlette'] = (df['diff_Option'] - df['Barlette Delta']*df['diff_SPX'])**2
        return 1 - df['sse_sabr'].sum() / df['sse_bs'].sum(), 1-df['sse_barlette'].sum() / df['sse_bs'].sum(),  1-df['sse_barlette'].sum() / df['sse_sabr'].sum()
    else:
        df['sse_bs'] = df['diff_Option'] - df['delta']*df['diff_SPX']
        df['sse_sabr'] = df['diff_Option'] - df['SABR Delta']*df['diff_SPX']
        df['sse_barlette'] = df['diff_Option'] - df['Barlette Delta']*df['diff_SPX']
        return 1 - df['sse_sabr'].std() / df['sse_bs'].std(), 1-df['sse_barlette'].std() / df['sse_bs'].std(),  1-df['sse_barlette'].std() / df['sse_sabr'].std()
    
gain = res.groupby(['bin1', 'bin2']).apply(calc_gain).reset_index()
gain['SABR Delta'] = gain[0].apply(lambda x: x[0])
gain['Barlette Delta'] = gain[0].apply(lambda x: x[1])
gain['Relative Gain Barlette'] = gain[0].apply(lambda x: x[2])
gain.drop(columns=0, inplace=True)
gain

,bin1,bin2,SABR Delta,Barlette Delta,Relative Gain Barlette
0,0,0,0.001250,0.001320,0.000070
1,0,1,0.063391,0.063501,0.000117
2,0,2,0.137840,0.137988,0.000172
3,0,3,0.135647,0.135772,0.000145
4,0,4,0.157850,0.158005,0.000184
...,...,...,...,...,...
58,8,2,0.013878,0.013909,0.000032
59,8,3,0.011915,0.011944,0.000028
60,8,4,0.013235,0.013266,0.000031
61,8,5,0.016252,0.016286,0.000035


In [30]:
gain_sabr = gain.pivot(index='bin1', columns='bin2', values='SABR Delta')
gain_sabr.index.name = 'Delta Bin'
gain_sabr.columns.name = 'Maturity Bin'
gain_sabr

Maturity Bin,0,1,2,3,4,5,6
Delta Bin,,,,,,,
0,0.001250,0.063391,0.137840,0.135647,0.157850,0.203718,0.227222
1,0.062549,0.099877,0.160585,0.151616,0.158889,0.168028,0.207184
2,0.069542,0.092356,0.140996,0.128759,0.147668,0.167442,0.172312
3,0.053505,0.069989,0.108657,0.093289,0.113394,0.122372,0.142038
4,0.030652,0.045620,0.066670,0.066892,0.081166,0.087200,0.097666
5,0.012801,0.026738,0.037544,0.042093,0.054104,0.059628,0.060744
6,0.003566,0.016322,0.024400,0.028802,0.034608,0.039754,0.042471
7,0.002134,0.011711,0.018140,0.019267,0.022931,0.027646,0.027224
8,0.005962,0.009124,0.013878,0.011915,0.013235,0.016252,0.012714


In [31]:
(gain_sabr>0).sum().sum()

63

Both SABR delta and Bartlett delta provide consistently better hedges than BS delta for all moneyness and maturity buckets.

In [32]:
# plot the gain of SABR Delta
fig = go.Figure(data=go.Heatmap(
                   z=gain_sabr.values,
                   x=gain_sabr.columns,
                   y=gain_sabr.index,
                   colorscale='Viridis'))
fig.update_layout(title='SABR Delta Gain', xaxis_title='Maturity', yaxis_title='BS Delta', width=800, height=500)
iplot(fig)

In [33]:
gain_barlette = gain.pivot(index='bin1', columns='bin2', values='Barlette Delta')
gain_barlette

bin2,0,1,2,3,4,5,6
bin1,,,,,,,
0,0.001320,0.063501,0.137988,0.135772,0.158005,0.203901,0.227368
1,0.062606,0.099966,0.160710,0.151735,0.159019,0.168144,0.207316
2,0.069600,0.092444,0.141115,0.128874,0.147792,0.167570,0.172433
3,0.053564,0.070075,0.108777,0.093395,0.113509,0.122490,0.142162
4,0.030708,0.045699,0.066779,0.066988,0.081269,0.087306,0.097778
5,0.012849,0.026803,0.037632,0.042171,0.054193,0.059721,0.060831
6,0.003606,0.016377,0.024471,0.028866,0.034680,0.039830,0.042547
7,0.002164,0.011751,0.018192,0.019314,0.022983,0.027703,0.027279
8,0.005981,0.009147,0.013909,0.011944,0.013266,0.016286,0.012746


In [34]:
(gain_barlette>0).sum().sum()

63

In [35]:
# plot the gain of Barlette Delta
fig = go.Figure(data=go.Heatmap(
                   z=gain_barlette.values,
                   x=gain_barlette.columns,
                   y=gain_barlette.index,
                   colorscale='Viridis'))
fig.update_layout(title='Barlette Delta Gain', xaxis_title='Maturity', yaxis_title='BS Delta', width=800, height=500)
iplot(fig)

Annotate the plot with the Gain values for each bucket.

In [54]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplot with 2D heatmap
fig = make_subplots(rows=1, cols=1)

# Add heatmap data
fig = go.Figure(data=go.Heatmap(
                   z=gain_barlette.values,
                   x=gain_barlette.columns,
                   y=gain_barlette.index,
                   colorscale='Viridis'))

# Add text in each cell over heatmap
for i in range(len(gain_barlette.index)):
    for j in range(len(gain_barlette.columns)):
        fig.add_annotation(dict(font=dict(color="white",size=14),
                                x=gain_barlette.columns[j],
                                y=gain_barlette.index[i],
                                text=str(round(gain_barlette.values[i][j], 3)),
                                showarrow=False))

fig.update_layout(title='Barlette Delta Gain', xaxis_title='Maturity', yaxis_title='BS Delta', width=800, height=500)
iplot(fig)


#### **Hedging Effectiveness: $\delta_{\text{SABR}}$ vs $\delta_{\text{Bartlett}}$**

Finally, we compare the hedging performance between SABR delta and Bartlett delta for each bucket.

SABR delta may lead to different conventional hedges especially near the money, yet Bartlett's delta is nearly independent of $\beta$ and tends to provide more robust hedges especially for near the money options. We calculate the relative Gain of Bartlett's delta over SABR delta for each bucket.

$$\text{Relative Gain} = 1- \cfrac{\sum(\Delta f  - \delta_{\text{Bartlett}}\Delta S)^2}{\sum(\Delta f  - \delta_{\text{SABR}}\Delta S)^2}$$

In [37]:
gain_relative = gain.pivot(index='bin1', columns='bin2', values='Relative Gain Barlette')
gain_relative

bin2,0,1,2,3,4,5,6
bin1,,,,,,,
0,0.000070,0.000117,0.000172,0.000145,0.000184,0.000230,0.000189
1,0.000061,0.000099,0.000149,0.000141,0.000155,0.000138,0.000166
2,0.000062,0.000097,0.000138,0.000131,0.000145,0.000153,0.000147
3,0.000062,0.000093,0.000134,0.000116,0.000129,0.000134,0.000144
4,0.000058,0.000083,0.000117,0.000103,0.000113,0.000116,0.000124
5,0.000049,0.000067,0.000091,0.000081,0.000094,0.000099,0.000092
6,0.000040,0.000055,0.000073,0.000066,0.000074,0.000080,0.000079
7,0.000030,0.000040,0.000054,0.000047,0.000053,0.000059,0.000057
8,0.000019,0.000024,0.000032,0.000028,0.000031,0.000035,0.000032


In [38]:
# plot the relative gain of Barlette Delta
fig = go.Figure(data=go.Heatmap(
                   z=gain_relative.values,
                   x=gain_relative.columns,
                   y=gain_relative.index,
                   colorscale='RdBu'))
fig.update_layout(title='Relative Gain Barlette Delta', xaxis_title='Maturity', yaxis_title='BS Delta', width=800, height=500)
iplot(fig)

In [53]:
# plot the relative gain of Barlette Delta
fig = go.Figure(data=go.Heatmap(
                   z=gain_relative.values,
                   x=gain_relative.columns,
                   y=gain_relative.index,
                   colorscale='Cividis',
                   hoverongaps = False))

annotations = []
for i, row in enumerate(gain_relative.values):
    for j, val in enumerate(row):
        annotations.append(go.layout.Annotation(text=str(round(val*1e6, 2))+'u', x=gain_relative.columns[j], y=gain_relative.index[i], font=dict(color='white'), showarrow=False))

fig.update_layout(title='Relative Gain Barlette Delta', xaxis_title='Maturity', yaxis_title='BS Delta', width=800, height=500, annotations=annotations)
iplot(fig)

Empirical results show:
- Bartlett's delta performs slightly but consistently better than SABR delta, giving the optimal delta hedging in the SABR model.
- Bartlett's delta performs uniformly better especially for near the money options across all maturities.